# Week 05 Live Coding Demo: OOP & Best Practices
Flow: functional pipeline (cons) → one larger OOP example → errors & exceptions → PEP 8 essentials.

## 1. Functional programming pipeline — thermistor processing

In [ ]:
def counts_to_voltage(counts, v_ref, n_bits):
    return [c * (v_ref / (2**n_bits - 1)) for c in counts]

def voltage_to_tempC(volts, a, b, c):
    return [a + b*v + c*(v*v) for v in volts]

def moving_avg(seq, win):
    half = win // 2
    out = []
    for i in range(len(seq)):
        lo, hi = max(0, i-half), min(len(seq), i+half+1)
        out.append(sum(seq[lo:hi]) / (hi - lo))
    return out

def crossings_above(seq, thresh, min_separation):
    idxs = []
    last = -1e9
    for i, v in enumerate(seq):
        if v > thresh and (i - last) >= min_separation:
            idxs.append(i); last = i
    return idxs

def summarize(idxs, sample_rate_hz, label):
    count = len(idxs)
    rate = count * sample_rate_hz /  len_t  # uses global-ish len_t (fragile)
    return {"label": label, "count": count, "events_per_s": rate}

len_t = 1000
raw_counts = [0]*400 + [50]*50 + [0]*100 + [60]*50 + [0]*400
v_ref, n_bits = 3.3, 8
a, b, c = 20.0, 8.0, -1.5
win = 21; thresh_C = 25.0; min_sep = 40; fs = 100.0

volts   = counts_to_voltage(raw_counts, v_ref, n_bits)
temps_C = voltage_to_tempC(volts, a, b, c)
smooth  = moving_avg(temps_C, win)
idx     = crossings_above(smooth, thresh_C, min_sep)
report  = summarize(idx, fs, "thermistor-A")

print("volts[440:455] =", [round(v,4) for v in volts[440:455]])
print("temps_C[440:455] =", [round(t,3) for t in temps_C[440:455]])
print("first indices over threshold:", idx[:5])
print("report:", report)


## 2. OOP: a single, larger class — `Spectrum1D`

In [ ]:
import numpy as np

class Spectrum1D:
    """
    Represents a 1-dimensional spectrum with wavelength [nm] and counts [a.u.].
    Attributes:
        wavelength_nm (ndarray): Wavelength values in nanometers.
        counts (ndarray): Corresponding count values.
        gain (float): Gain factor.
        offset (float): Offset value.
        integration_s (float): Integration time in seconds.

    Methods:
        subtract_baseline: Subtract a baseline from the spectrum.
        normalize: Normalize the spectrum.
        window: Select a wavelength range from the spectrum.
        to_photon_flux_per_s: Convert the spectrum to photon flux per second.
        is_close: Check if two spectra are close.

    Examples:
        >>> spec = Spectrum1D(wl, I, gain=2.0, offset=0.02, integration_s=0.5)
        >>> print(spec.peak_wavelength_nm)
        >>> spec2 = spec.window(520, 600)
        >>> print(spec2.to_photon_flux_per_s(efficiency=0.5))
    """
    DEFAULT_UNITS = ("nm", "counts")
    def __init__(self, wavelength_nm, counts, *, gain=1.0, offset=0.0, integration_s=1.0): # * means keyword-only args to the right
        wl = np.asarray(wavelength_nm, dtype=float)
        I  = np.asarray(counts, dtype=float)
        if wl.shape != I.shape: raise ValueError("wavelength and counts must have the same shape")
        if wl.ndim != 1: raise ValueError("spectrum must be 1-D")
        if np.any(I < 0): raise ValueError("counts must be nonnegative")
        if integration_s <= 0: raise ValueError("integration_s must be positive")
        self._wl = wl; self._I = I
        self.gain = float(gain); self.offset = float(offset); self.integration_s = float(integration_s)
        self._sorted = np.all(np.diff(self._wl) >= 0)
    def __repr__(self):
        return (f"Spectrum1D(N={self._wl.size}, range=[{self._wl.min():.1f},{self._wl.max():.1f}] nm, "
                f"units={self.DEFAULT_UNITS})")
    def __str__(self):
        return f"Spectrum1D with {self._wl.size} points, {self.DEFAULT_UNITS[0]}/{self.DEFAULT_UNITS[1]}"
    def _ensure_sorted(self):
        if not self._sorted:
            order = np.argsort(self._wl)
            self._wl = self._wl[order]; self._I = self._I[order]; self._sorted = True
    @property
    def wavelength_nm(self): return self._wl.copy() # Make a copy to prevent external modification
    @property
    def counts(self): return self.gain * (self._I - self.offset) # Make a copy to prevent external modification
    @property
    def peak_wavelength_nm(self):
        i = int(np.argmax(self.counts)); return float(self._wl[i])
    @property
    def snr(self):
        I = self.counts
        if I.size < 10: return float("nan")
        k = max(1, I.size // 10)
        top = np.mean(np.sort(I)[-k:]); bottom = np.std(np.sort(I)[:k]) + 1e-12
        return float(top / bottom)
    @property
    def bandwidth_fwhm_nm(self):
        """Full width at half maximum [nm], baseline-aware with linear interpolation."""  # concise
        self._ensure_sorted()  # ensure wavelength is monotonic
        I = self.counts
        if I.size < 3 or not np.all(np.isfinite(I)): return float("nan")  # guard

        imax = int(np.argmax(I)); p = I[imax]  # peak index and value
        bkg = float(np.percentile(I, 5))       # robust baseline (bottom 5th percentile)
        level = bkg + 0.5 * (p - bkg)          # half-maximum above baseline

        # search left crossing (I rises through 'level')
        li = imax
        while li > 0 and I[li] > level: li -= 1
        if li == 0 and I[li] > level: return float("nan")  # no left crossing
        # linear interpolation between (li, li+1)
        li2 = min(li + 1, I.size - 1)
        denomL = (I[li2] - I[li]) or 1e-12
        wl_left = self._wl[li] + (level - I[li]) * (self._wl[li2] - self._wl[li]) / denomL  # interp

        # search right crossing (I falls through 'level')
        ri = imax
        n = I.size
        while ri < n - 1 and I[ri] > level: ri += 1
        if ri == n - 1 and I[ri] > level: return float("nan")  # no right crossing
        # linear interpolation between (ri-1, ri)
        ri1 = max(ri - 1, 0)
        denomR = (I[ri] - I[ri1]) or 1e-12
        wl_right = self._wl[ri1] + (level - I[ri1]) * (self._wl[ri] - self._wl[ri1]) / denomR  # interp

        return float(wl_right - wl_left)  # width in nm

    def subtract_baseline(self, method="median", value=None):
        if method == "median" and value is None:
            b = float(np.median(self._I))
        elif method == "value" and value is not None:
            b = float(value)
        else:
            raise ValueError("method must be 'median' or 'value' with value provided")
        self._I = self._I - b; return self
    def normalize(self, method="max"):
        if method == "max":
            s = float(np.max(np.abs(self._I))) or 1.0
        elif method == "area":
            s = float(np.trapezoid(np.abs(self._I), self._wl)) or 1.0
        else:
            raise ValueError("method must be 'max' or 'area'")
        self._I = self._I / s; return self
    def window(self, wl_min, wl_max):
        self._ensure_sorted(); m = (self._wl >= wl_min) & (self._wl <= wl_max)
        self._wl = self._wl[m]; self._I = self._I[m]; return self
    def is_close(self, other, rtol=1e-6, atol=1e-9):
        return (np.allclose(self.wavelength_nm, other.wavelength_nm, rtol=rtol, atol=atol) and
                np.allclose(self.counts, other.counts, rtol=rtol, atol=atol))
    def to_photon_flux_per_s(self, efficiency=1.0):
        h = 6.62607015e-34; c = 2.99792458e8
        E = h*c / (self._wl*1e-9)
        photons = (self.counts / efficiency)
        return np.trapezoid(photons, self._wl) / max(self.integration_s, 1e-12)

import matplotlib.pyplot as plt
wl = np.linspace(480, 640, 801)
true_peak, width = 560.0, 12.0
I = np.exp(-0.3*((wl-true_peak)/width)**2) + 0.05*np.sin(0.5*wl) + 0.01*np.random.RandomState(0).randn(wl.size) + 10
plt.plot(wl, I, label="raw counts"); plt.show()
spec = Spectrum1D(wl, I, gain=2.0, offset=0.02, integration_s=0.5)

print(repr(spec))
print(str(spec))
print("peak_wavelength_nm:", round(spec.peak_wavelength_nm, 2))
print("SNR:", round(spec.snr, 2))
print("FWHM estimate:", round(spec.bandwidth_fwhm_nm, 2))

spec2 = Spectrum1D(wl, I).subtract_baseline("median").normalize("max").window(520, 600)
print("windowed N:", spec2.wavelength_nm.size)
print("photon flux (a.u.):", round(spec2.to_photon_flux_per_s(efficiency=0.5), 4))
print("close?", spec2.is_close(spec2))


## 3. Common error types — caught so the notebook continues

In [ ]:
import math
def demo_exception(label, func):
    try:
        func()
    except Exception as e:
        print(f"{label}: {type(e).__name__}: {e}")
cases = [
    ("SyntaxError", lambda: compile("if x = 3\n    pass", "<demo>", "exec")),
    ("IndentationError", lambda: compile("def f():\nprint('x')", "<demo>", "exec")),
    ("NameError", lambda: eval("freq_Hz")),
    ("TypeError", lambda: (3 + "eV")),
    ("ValueError", lambda: int("pi")),
    ("IndexError", lambda: [1,2,3][5]),
    ("KeyError", lambda: {"a":1}["b"]),
    ("AttributeError", lambda: "hello".append("!")),
    ("ZeroDivisionError", lambda: 1/0),
    ("OverflowError", lambda: math.exp(1000)),
]
for label, fn in cases:
    demo_exception(label, fn)


## 4. Exceptions: try / except / else / finally + assertions

In [ ]:
def safe_log_ratio(a, b):
    """Return log(a/b); requires a>0 and b>=0; b==0 → ±inf. Demonstrates try/except/else/finally."""  # contract
    # ---- Contract validation (predictable) ----
    if a <= 0: raise ValueError("a must be > 0")                    # guard for domain of log
    if b < 0:  raise ValueError("b must be >= 0")                   # guard for domain of log (allow b==0)
    # Optional: assert scalar inputs to show assertions in the section title
    assert np.isscalar(a) and np.isscalar(b), "a, b must be scalars"  # assertion for developer assumptions

    # ---- Risky operation (runtime failure) ----
    try:
        q = a / b                                                   # may raise ZeroDivisionError
    except ZeroDivisionError:
        return math.copysign(float("inf"), a)                       # define behavior at b==0
    else:
        return math.log(q)                                          # runs only if no exception in try
    finally:
        print(f"[audit] safe_log_ratio called with a={a}, b={b}")   # always runs
      
print(round(safe_log_ratio(10.0, 2.0), 6))
try:
    print(safe_log_ratio(-1.0, 2.0))
except Exception as e:
    print("caught:", type(e).__name__, str(e))
print(safe_log_ratio(1.0, 0.0))

In [ ]:
def period_small_angle_pendulum(L_m, g=9.81):
    if not isinstance(L_m, (int, float)) or L_m <= 0:
        raise ValueError("L_m must be a positive number")
    omega2 = g / L_m
    assert omega2 > 0, "internal invariant: omega^2 must be > 0"
    import math
    return 2*math.pi / math.sqrt(omega2)
print(round(period_small_angle_pendulum(1.0, g=10), 6))

## 5. PEP 8 essentials — naming, spaces, docstrings vs comments

In [ ]:
# =========================
# BAD vs GOOD — PEP 8 demo
# =========================

# ---------- BAD EXAMPLES ----------
# Naming: constant not ALL_CAPS and overly long float literal w/o underscores (hard to read)
speedOfLight = 299792458.0

# Naming: function & vars not snake_case; Whitespace: inconsistent spaces around operators and commas
def LorentzGamma(v): # BAD: CapWords for function name; no docstring
    from math import sqrt     # Imports: should be at top; local import OK but not the rule
    if(not(0<=v and v<speedOfLight)):   # BAD: extra spaces missing/parentheses clutter; odd comparison
        raise ValueError("speed must be in [0,c)")
    return 1.0/sqrt(1.0-(v/speedOfLight)**2) # BAD: crowding around operators

# Whitespace in function args: BAD spacing — spaces around '=' in defaults; missing after commas
def add_bad(a =1,b =2,c= 3+4):  # BAD: spaces around '=' in defaults; no space after commas
    return a+b+c                # BAD: crowded operators

# Comments vs docstrings: comment used instead of docstring (undiscoverable by help())
def snell_n2_bad(n1,t1,t2): # BAD: missing spaces after commas; vague names
    # Return n2 using Snell's law n1 sin(t1) = n2 sin(t2)  (comment, not a docstring)
    import math
    return n1*math.sin(t1)/max(1e-16,math.sin(t2)) # BAD: crowded ops, no spaces after commas

# Naming: class not CapWords; Docstring missing
class relativistic_particle: # BAD: class should be CapWords
    def __init__(self, v_m_s):
        if not(0<=v_m_s<speedOfLight):  # BAD: crowded comparisons, extra parens
            raise ValueError("Velocity must be in [0,c)")
        self.v=v_m_s                    # BAD: non-descriptive attribute
    def Gamma(self):                    # BAD: method not snake_case; no docstring
        return LorentzGamma(self.v)

# Quick use to show it still runs (but style is poor)
print("BAD gamma:", round(LorentzGamma(1e7), 6))
print("BAD add:", add_bad(1,2,3))


# ---------- GOOD EXAMPLES ----------
# Naming
# Constants: UPPER_CASE_WITH_UNDERSCORES → SPEED_OF_LIGHT_M_S
SPEED_OF_LIGHT_M_S = 2.997_924_58e8  # meters/second

# Imports
# Normally at top, but local import can be justified if only used rarely (to save load time).

# Naming
# Functions & variables: snake_case → lorentz_gamma, speed_m_s
def lorentz_gamma(speed_m_s):
    # Docstrings
    # Triple-quoted """ right under def or class. Used to describe purpose/params/returns; discoverable via help().
    """
    Return Lorentz gamma = 1/sqrt(1 - (v/c)^2).

    Parameters
    ----------
    speed_m_s : float
        Speed in meters per second, must satisfy 0 <= v < c.

    Returns
    -------
    float
        Lorentz factor gamma.
    """
    from math import sqrt  # local import acceptable here for a rarely used function
    v = float(speed_m_s)   # clear variable name in snake_case

    # Whitespace / operators
    # Spaces around =, arithmetic, comparisons; no extra spaces inside parentheses.
    # if not (0 <= v < SPEED_OF_LIGHT_M_S):
    if not (0 <= v < SPEED_OF_LIGHT_M_S):
        raise ValueError("speed must be in [0, c)")

    return 1.0 / sqrt(1.0 - (v / SPEED_OF_LIGHT_M_S) ** 2)

# Whitespace in function args: demo the *lack* of spaces around '=' in defaults (PEP 8), but keep spaces around '+'
def add_good(a=1, b=2, c=3 + 4):  # GOOD: no spaces around '=' in defaults; spaces after commas; spaces around '+'
    return a + b + c              # GOOD: spaces around operators

def snell_n2(n1, theta1_rad, theta2_rad):
    """
    Return n2 from Snell's law: n1 * sin(theta1) = n2 * sin(theta2).

    Parameters
    ----------
    n1 : float
        Refractive index of medium 1.
    theta1_rad : float
        Incident angle [rad].
    theta2_rad : float
        Refracted angle [rad].

    Returns
    -------
    float
        Refractive index of medium 2.
    """
    import math  # local import fine here
    return n1 * math.sin(theta1_rad) / max(1e-16, math.sin(theta2_rad))

# Naming
# Classes: CapWords → RelativisticParticle
class RelativisticParticle:
    """
    Simple particle with velocity and Lorentz gamma factor.

    Attributes
    ----------
    v_m_s : float
        Velocity [m/s].
    """
    def __init__(self, v_m_s):
        if not (0 <= v_m_s < SPEED_OF_LIGHT_M_S):  # GOOD: clean comparison, spacing
            raise ValueError("Velocity must be in [0, c)")
        self.v_m_s = float(v_m_s)  # clear attribute name

    def gamma(self):
        """Return Lorentz gamma for this particle."""
        return lorentz_gamma(self.v_m_s)

# Comments
# Start with #, inline or above a line of code. Not discoverable by help().
# (Docstrings above *are* discoverable: try help(snell_n2).)

# Good usages
print("GOOD gamma:", round(lorentz_gamma(1e7), 6))        # example call
print("GOOD add:", add_good())                            # uses defaults (note arg spacing rule)
p = RelativisticParticle(1e7)                             # class instantiation
print("GOOD gamma (class):", round(p.gamma(), 6))         # method call
print("docstring available?", snell_n2.__doc__ is not None)  # True
